In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from tqdm import tqdm
import akshare as ak
import datetime
from multiprocessing import Pool
from sklearn.metrics import confusion_matrix

ModuleNotFoundError: No module named 'akshare'

In [ ]:
# 获取2020-01-01 至 今天 所有A股信息

# 获取股票列表
today_stock = ak.stock_zh_a_spot_em()[['代码']]
stock_code_list = sorted(today_stock['代码'].to_list())

# 股票原始数据API
def raw_stock_data_api(stock_code):
    return  ak.stock_zh_a_hist(symbol=stock_code, period="daily", start_date='20000101', end_date='20250306', adjust="")

# 股票复权数据API
def hfq_stock_data_api(stock_code):
    return ak.stock_zh_a_hist(symbol=stock_code, period="daily", start_date='20000101', end_date='20250306', adjust="hfq")

# 获取并处理股票原始数据
def get_raw_stock_data(stock_code_list):
    # 读取数据
    all_stock_data = []
    with Pool() as pool:
        results = list(tqdm(pool.imap(raw_stock_data_api, stock_code_list), total=len(stock_code_list)))
    for data in results:
        all_stock_data.append(data)
    stock_data = pd.concat(all_stock_data,axis=0)

    # 清洗数据
    stock_data = stock_data[['日期','股票代码','开盘','收盘','最高','最低']]
    stock_data.rename(columns={'日期':'date','股票代码':'stockcode','开盘':'openprice','收盘':'closeprice','最高':'highprice','最低':'lowprice'},inplace=True)
    stock_data['date'] = stock_data['date'].astype(str)
    stock_data.set_index(['date','stockcode'],inplace=True)
    stock_data.sort_index(inplace=True)

    return stock_data

# 获取并处理股票复权数据
def get_hfq_stock_data(stock_code_list):
    # 读取数据
    all_stock_data = []
    with Pool() as pool:
        results = list(tqdm(pool.imap(hfq_stock_data_api, stock_code_list), total=len(stock_code_list)))
    for data in results:
        all_stock_data.append(data)
    stock_data = pd.concat(all_stock_data,axis=0)

    # 清洗数据
    stock_data = stock_data[['日期','股票代码','开盘','收盘','最高','最低','成交量','成交额','换手率']]
    stock_data.rename(columns={'日期':'date','股票代码':'stockcode','成交量':'turnovervolumn','成交额':'turmovervalue','换手率':'turnoverrate',
                                '开盘':'backward_openprice','收盘':'backward_closeprice','最高':'backward_highprice','最低':'backward_lowprice'},inplace=True)
    stock_data['date'] = stock_data['date'].astype(str)
    stock_data.set_index(['date','stockcode'],inplace=True)
    stock_data.sort_index(inplace=True)

    return stock_data
raw_stock_data = get_raw_stock_data(stock_code_list)
hfq_stock_data = get_hfq_stock_data(stock_code_list)
stock_data = pd.merge(raw_stock_data,hfq_stock_data,left_index=True,right_index=True)

# 删掉复权价格小于0的股票
remove_stocks = stock_data[stock_data['backward_closeprice']<0].index.get_level_values(1).unique()
stock_data = stock_data.drop(index=remove_stocks, level=1)

In [ ]:
def feature_process(stock_code):
    one_stock = stock_data.loc[(slice(None),stock_code),].copy()
    if len(one_stock)<=100:
        return None

    # 计算移动平均线
    one_stock['MA_5']  = one_stock['backward_closeprice'].rolling(window=5,min_periods=5).mean()
    one_stock['MA_10'] = one_stock['backward_closeprice'].rolling(window=10,min_periods=10).mean()
    one_stock['MA_20'] = one_stock['backward_closeprice'].rolling(window=20,min_periods=20).mean()
    one_stock['MA_60'] = one_stock['backward_closeprice'].rolling(window=60,min_periods=60).mean()
    one_stock['MA_90'] = one_stock['backward_closeprice'].rolling(window=90,min_periods=90).mean()

    # 计算偏离值
    one_stock['bias_5']  = one_stock['backward_closeprice'] - one_stock['MA_5']
    one_stock['bias_10'] = one_stock['backward_closeprice'] - one_stock['MA_10']
    one_stock['bias_20'] = one_stock['backward_closeprice'] - one_stock['MA_20']
    one_stock['bias_60'] = one_stock['backward_closeprice'] - one_stock['MA_60']
    one_stock['bias_90'] = one_stock['backward_closeprice'] - one_stock['MA_90']

    # 计算标准差
    one_stock['std_5']  = one_stock['backward_closeprice'].rolling(window=5,min_periods=5).std()
    one_stock['std_10'] = one_stock['backward_closeprice'].rolling(window=10,min_periods=10).std()
    one_stock['std_20'] = one_stock['backward_closeprice'].rolling(window=20,min_periods=20).std()
    one_stock['std_60'] = one_stock['backward_closeprice'].rolling(window=60,min_periods=60).std()
    one_stock['std_90'] = one_stock['backward_closeprice'].rolling(window=90,min_periods=90).std()

    # 计算相对强弱指数（RSI）
    delta = one_stock['backward_closeprice'].diff(1)
    one_stock['RSI_10'] = (delta.where(delta > 0, 0)).rolling(window=14).mean() / (-delta.where(delta < 0, 0)).rolling(window=10).mean()
    one_stock['RSI_20'] = (delta.where(delta > 0, 0)).rolling(window=14).mean() / (-delta.where(delta < 0, 0)).rolling(window=20).mean()
    one_stock['RSI_60'] = (delta.where(delta > 0, 0)).rolling(window=14).mean() / (-delta.where(delta < 0, 0)).rolling(window=60).mean()
    one_stock['RSI_90'] = (delta.where(delta > 0, 0)).rolling(window=14).mean() / (-delta.where(delta < 0, 0)).rolling(window=90).mean()

    # 计算成交量变异系数（VCV）
    one_stock['VCV_10'] = one_stock['turnovervolumn'].rolling(10).std() / one_stock['turnovervolumn'].rolling(10).mean()
    one_stock['VCV_20'] = one_stock['turnovervolumn'].rolling(20).std() / one_stock['turnovervolumn'].rolling(20).mean()
    one_stock['VCV_60'] = one_stock['turnovervolumn'].rolling(60).std() / one_stock['turnovervolumn'].rolling(60).mean()
    one_stock['VCV_90'] = one_stock['turnovervolumn'].rolling(90).std() / one_stock['turnovervolumn'].rolling(90).mean()

    # 计算其他指标
    one_stock['VPT'] = (one_stock['turnovervolumn'] * (one_stock['backward_closeprice'].pct_change())).cumsum()
    one_stock['Price_Vol_Corr'] = one_stock['backward_closeprice'].pct_change().rolling(20).corr(one_stock['turnovervolumn'].pct_change())
    one_stock['OBV'] = (one_stock['turnovervolumn'] * one_stock['backward_closeprice'].pct_change().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))).cumsum()
    one_stock['VR_20'] = one_stock['turnovervolumn'].where(one_stock['backward_closeprice'].diff() > 0, 0).rolling(20).sum() / one_stock['turnovervolumn'].where(one_stock['backward_closeprice'].diff() < 0, 0).rolling(20).sum()
    one_stock['PAV_20'] = one_stock['turnovervolumn'] / one_stock['backward_closeprice'].rolling(20).mean()

    # 处理无穷值和缺失值
    one_stock = one_stock.iloc[100:,:]
    one_stock.replace([np.inf, -np.inf], np.nan, inplace=True)
    one_stock.fillna(one_stock.mean(), inplace=True)

    # 计算标签
    one_stock['5F_ret'] = np.log(one_stock['backward_closeprice']).shift(-5) - np.log(one_stock['backward_closeprice'])

    return one_stock

all_stock_data = []
stock_code_list = stock_data.index.get_level_values(1).unique()

with Pool() as pool:
    results = list(tqdm(pool.imap(feature_process, stock_code_list), total=len(stock_code_list)))
for data in results:
    all_stock_data.append(data)
stock_data = pd.concat(all_stock_data,axis=0)
stock_data.sort_index(inplace=True)

In [ ]:
def standard(x):
    x = (x - x.mean()) / x.std()
    return x

stock_data = stock_data.loc[:stock_data.index.get_level_values(0).unique()[-6],:]
stock_data_standardized = stock_data.groupby(level=0).transform(standard)
stock_data_standardized.dropna(inplace=True)

In [ ]:
import lightgbm as lgb

model = lgb.LGBMRegressor(
    boosting_type='gbdt',  # 梯度提升树
    num_leaves=31,         # 叶子数，控制模型复杂度
    learning_rate=0.05,    # 学习率
    n_estimators=100,      # 树的数量（迭代次数）
    subsample=0.8,         # 采样比例（防止过拟合）
    colsample_bytree=0.8,  # 特征采样比例
    verbosity = -1
)


date_list = stock_data_standardized.index.get_level_values(0).unique()
pred_list= []

for i in tqdm(range(0,len(date_list),30)):
    
    train_start_date = date_list[i]
    train_end_date = date_list[i+100]
    train_X = stock_data_standardized.loc[train_start_date:train_end_date,'openprice':'VCV_60']
    train_y = stock_data_standardized.loc[train_start_date:train_end_date,'5F_ret']
    
    if 135 + i >= len(date_list):
        test_X = stock_data_standardized.loc[test_start_date:,'openprice':'VCV_60']
        test_y = stock_data_standardized.loc[test_start_date:,'5F_ret']

        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        y_pred = pd.DataFrame({'pred':y_pred},index=test_y.index)
        pred_list.append(y_pred)
        break

    else:
        test_start_date = date_list[i+105]
        test_end_date = date_list[i+135]
        test_X = stock_data_standardized.loc[test_start_date:test_end_date,'openprice':'VCV_60']
        test_y = stock_data_standardized.loc[test_start_date:test_end_date,'5F_ret']

        model.fit(train_X, train_y)
        y_pred = model.predict(test_X)
        y_pred = pd.DataFrame({'pred':y_pred},index=test_y.index)
        pred_list.append(y_pred)

pred_result = pd.concat(pred_list)
pred_result['label'] = stock_data['5F_ret']
pred_result.dropna(inplace=True)

In [ ]:
date_list = pred_result.index.get_level_values(0).unique()
fund = 1
fund_list = []
ret_list = []

for date in tqdm(date_list):
    test_date = pred_result.loc[date,:]
    test_date = test_date.sort_values(by='pred').tail(20)
    
    ret = (test_date['label'].sum()) / (20 * 5)
    fund = fund + ret

    ret_list.append(ret)
    fund_list.append(fund)
fund_list /= fund_list[0]

In [17]:
PnL = pd.DataFrame({'strategy':fund_list,'ret':ret_list}, index=date_list)
index = ak.stock_zh_index_daily(symbol="sh000001")
index['date'] = index['date'].astype(str)
index.set_index(['date'],inplace=True)
PnL['price'] = index['close']
PnL['price'] = PnL['price']/ PnL.loc[PnL.index[0],'price']

In [ ]:
# 计算回测指标
returns = PnL['strategy'].pct_change().dropna()  # 计算每日收益率
annual_return = (PnL['strategy'].iloc[-1] / PnL['strategy'].iloc[0]) ** (252 / len(PnL)) - 1
annual_volatility = returns.std() * np.sqrt(252)
cum_max = PnL['strategy'].cummax()  # 计算累计最大净值
drawdown = PnL['strategy'] / cum_max - 1  # 计算回撤
max_drawdown = drawdown.min()  # 取最大回撤
sharpe_ratio = annual_return / annual_volatility if annual_volatility != 0 else np.nan

# 策略结果可视化
fig, ax1 = plt.subplots(figsize=(20, 6))
df = PnL.copy(deep=True)
df.index = pd.to_datetime(df.index)

# 折线图
ax1.plot(df.index.get_level_values('date'), df['price'], label='price', color='tab:blue', linewidth=1)
ax1.plot(df.index.get_level_values('date'), df['strategy'], label='strategy', color='tab:green', linestyle='dashed')

# 添加文本信息
textstr = f"annual_return: {annual_return:.2%}\nannual_volatility: {annual_volatility:.2%}\nmax_drawdown: {max_drawdown:.2%}\nsharpe_ratio: {sharpe_ratio:.2f}"
ax1.text(df.index.get_level_values('date')[-int(len(df) * 0.2)], df['price'].max() * 0.9, textstr,
        fontsize=12, bbox=dict(facecolor='white', alpha=0.6))

ax1.set_xlabel('date')
ax1.set_ylabel('net value')
ax1.set_title(f'return')
ax1.legend()
plt.grid(alpha=0.5)
plt.show()